In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_class = 2
ROWS, COLS = 224, 224
BATCHSIZE = 64
SEAL_CLASSES = ['NoS', 'seal']
nb_perClass = int(BATCHSIZE / len(SEAL_CLASSES))
TRAIN_DIR = '../darknet/seals/JPEGImagesBlk'
TEST_DIR = '../darknet/seals/JPEGImagesTest'


In [3]:
# Data generator
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True)

# Fine-tune Example
rfcnCV = pd.read_pickle('../coords/rfcnCV_small.pkl')
rfcnCV.head(3)

,img,proba,x0,y0,x1,y1,seal
0,228_54,0.745,435.35,390.8,535.35,490.8,0
1,236_22,0.791,435.60,129.5,535.60,229.5,0
2,536_64,0.734,406.55,444.0,506.55,544.0,0


In [ ]:
# Lets make our validation set
folds = [rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 != 0,
        rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 == 0]
rfcnCV.shape

(260268, 7)

In [ ]:
for fold in range(2):
    print "Fold: " + str(fold+1)
    train_df = rfcnCV[~folds[fold]]
    test_df = rfcnCV[folds[fold]]
    valid_df = test_df[:1500]
    
    # validation_data (valid_x,valid_y)
    print "Part ... A"
    df_1 = valid_df
    l = valid_df.groupby('seal').size()
    nb_NoF_valid = math.ceil(l.sum()/10)
    valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
    valid_y = np.zeros((valid_df.shape[0], len(SEAL_CLASSES)), dtype=K.floatx())
    i = 0
    for index,row in valid_df.iterrows():
        if index % 300 == 0 : print index
        row = row.tolist()
        image_file = os.path.join(TRAIN_DIR, row[0])
        seal = row[6]
        bbox = row[2:6]
        cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
        x = np.asarray(cropped, dtype=K.floatx())
        x = preprocess_input(x)
        valid_x[i] = x
        valid_y[i,seal] = 1
        i += 1
    valid_x = valid_x.transpose(0, 3, 1, 2)
    valid_x.shape

    # Load our model
    print "Part ... B"
    nb_epoch = 2
    samples_per_epoch = 70000
    model = vgg_std16_model(ROWS, COLS, channel, num_class)

    # Start Fine-tuning
    print "Part ... C"
    model.fit_generator(train_generator(train_datagen, train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch, #50000,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    for layer in model.layers[10:]:
        layer.trainable = True
    model.optimizer.lr = 1e-4
    nb_epoch = 2
    print "Part ... D"
    model.fit_generator(train_generator(train_datagen, df=train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    # Test preds save
    print "Part ... E"
    test_preds = model.predict_generator(testcv_generator(test_df), val_samples=test_df.shape[0])
    df = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(test_preds,  columns=['predNoSeal', 'predSeal'])], axis=1)
    df.to_pickle('../coords/vggCVPredssmall2105_fold' + str(fold+1) + '.pkl')
    df[['img', 'predSeal']].to_csv('../coords/vggCVPredssmall2105_fold' + str(fold+1) + '.csv', index=False)
    
    # Clean up
    del model, train_df, test_df, valid_df, valid_x, df_1
    gc.collect()
    gc.collect()

Fold: 1
Part ... A
103200
103500
103800
104100
104400
Part ... B
Part ... C
Epoch 1/2
  384/70000 [..............................] - ETA: 11514s - loss: 0.8359 - acc: 0.6276